In [ ]:
# Importing necessary libraries
from matplotlib import legend
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import os
import numpy as np
from icecream import ic
from tqdm import tqdm
from itertools import chain
import logging
import os
import cProfile
import pstats
#from dataloader import *
from get_data import *
from torch.utils.data import Dataset, DataLoader
from dataloader import *

In [ ]:
visualise(get_data(path = "save_data_test4.csv", timesteps_from_data=0, skip_steps_start = 0, skip_steps_end = 0, 
                   drop_half_timesteps = False, normalise_s_w=False, rescale_p=False, num_inits=100), num_inits=100, set_ylim=True)

In [180]:
input_data = get_data(path = "save_data_test4.csv", 
                        timesteps_from_data=0, 
                        skip_steps_start = 0,
                        skip_steps_end = 0, 
                        drop_half_timesteps = False,
                        normalise_s_w=True,
                        rescale_p=False,
                        num_inits=4)

cut_off_timesteps = 800

#Split data into train and test sets
percentage_of_data = 0.8
window_size = 15
batch_size = 2
future=6

num_of_inits_train = int(len(input_data)*percentage_of_data)
train_inits = np.random.randint(0,len(input_data), num_of_inits_train)
train_inits = np.unique(train_inits)
test_inits = np.array([x for x in range(len(input_data)) if x not in train_inits])

# make sure we really get the specified percentage of training data..
if percentage_of_data < 0.99: 
        while len(train_inits) < num_of_inits_train:
            i = np.random.randint(0,len(test_inits),1)[0]
            train_inits = np.append(train_inits,test_inits[i])
            test_inits = np.delete(test_inits, i)

train_data = input_data[train_inits,:input_data.size(dim=1)-cut_off_timesteps,:]
test_data = input_data[test_inits,:,:]

data_set  = CustomDataset(train_data, window_size=window_size, future=future)
train_dataloader = DataLoader(data_set, batch_size=batch_size)#, drop_last=True)

In [181]:
print(train_data.size())
print((train_data.size(dim=0)*train_data.size(dim=1))/(batch_size))
for k, (x,y) in enumerate(train_dataloader):
    #print(y, y.size())
   # print(x[0,-1,0] , y[0,0,0])
    if x[-1,-1,0]  == x[-1,-2,0]:
        print(2*k+window_size,"skip")
    #print("label:",y)
    #print("timestep mit label simma bei t = ", k, ":", k+(window_size+2))
    #print("-----------------------------")
    #print(k)


torch.Size([3, 300, 3])
450.0
301 skip
303 skip
305 skip
307 skip
309 skip
311 skip
313 skip
601 skip
603 skip
605 skip
607 skip
609 skip
611 skip
613 skip


In [149]:
def visualise_batching(train_dataloader, train_data):

    n  = 10*train_data.size(dim=1)

    timesteps = np.linspace(0,n,n)

    for k, (x, y) in enumerate(train_dataloader):
        # print(x,x.size(), timesteps[k*window_size:(k+1)*window_size])
        #if (k+1)*window_size < 10*train_data.size(dim=1):
        for b in range(batch_size):
         if b%10==0:
            plt.plot(timesteps[k*batch_size+b: k*batch_size+b+window_size], x[b,:,1].view(window_size).numpy(), alpha=0.2, linewidth=5, color="red")
            plt.plot(timesteps[k*batch_size+b+window_size:k*batch_size+b+window_size+future], y[b,:,1].view(future).numpy(), alpha=0.5, linewidth=5, color="green", linestyle="dotted")
            plt.plot(timesteps[k*batch_size+b: k*batch_size+b+window_size], x[b,:,1].view(window_size).numpy() - b, alpha=0.2, linewidth=5, color="red")
            plt.plot(timesteps[k*batch_size+b+window_size:k*batch_size+b+window_size+future], y[b,:,1].view(future).numpy() - b, alpha=0.5, linewidth=5, color="green", linestyle="dotted")
         #else:
            #plt.plot(timesteps[k*batch_size+b: k*batch_size+b+window_size], x[b,:,1].view(window_size).numpy(), alpha=0.2, linewidth=5, color="blue")
            #plt.plot(timesteps[k*batch_size+b+window_size:k*batch_size+b+window_size+future], y[b,:,1].view(future).numpy(), alpha=0.5, linewidth=5)

    plt.grid(True)
    plt.show()
    
%matplotlib qt 
visualise_batching(train_dataloader, train_data)

IndexError: index 20 is out of bounds for dimension 0 with size 20